Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [2]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [3]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [4]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [5]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [6]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [7]:
tokens = gather_data(path)

In [9]:
tokens[1][0:10]

['feels',
 'like',
 'snow',
 'place',
 'party',
 'try',
 'day',
 'dissuade',
 'father',
 'venturing']

In [8]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

## Follow Along

In [10]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [13]:
len(titles) == len(tokens)

True

In [12]:
len(tokens)

813

### Author DataFrame


In [14]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [15]:
df.head()

,tokens
Austen_Emma0026,"[giving, fair, companion, account, yesterday, ..."
Austen_Emma0032,"[feels, like, snow, place, party, try, day, di..."
CBronte_Villette0086,"[pierced, opaque, blackness, stood, bougie, qu..."
CBronte_Jane0099,"[joke, queer, looks, tell, thing, specially, s..."
CBronte_Villette0092,"[second, landing, floor, comprising, abode, kn..."


In [16]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [17]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [18]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [19]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[giving, fair, companion, account, yesterday, ...",1,Emma,26
Austen_Emma0032,"[feels, like, snow, place, party, try, day, di...",1,Emma,32
CBronte_Villette0086,"[pierced, opaque, blackness, stood, bougie, qu...",0,Villette,86
CBronte_Jane0099,"[joke, queer, looks, tell, thing, specially, s...",0,Jane,99
CBronte_Villette0092,"[second, landing, floor, comprising, abode, kn...",0,Villette,92


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [20]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [21]:
streaming_data = doc_stream(path)

In [23]:
type(tokens)

list

In [22]:
type(streaming_data)

generator

In [ ]:
# gather_data => returns a list
# doc_stream => returns a generator

In [33]:
next(streaming_data)[:10] # Returns one document at a time from the generator

['nod',
 'smile',
 'approbation',
 'thought',
 'looking',
 'point',
 'unlikely',
 'engage',
 'considered',
 'dressed']

### Gensim LDA Topic Modeling

In [34]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [38]:
id2word.token2id['america']

15378

In [39]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(869, 3), (1254, 1), (2485, 1), (16850, 1)]

In [40]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

64
7064


In [56]:
len(id2word.keys())

22095

In [41]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=2, no_above=0.95)

In [42]:
len(id2word.keys())

14392

In [43]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [44]:
corpus[345][:10]

[(4, 1),
 (18, 1),
 (29, 1),
 (31, 1),
 (32, 2),
 (42, 1),
 (48, 1),
 (53, 1),
 (57, 1),
 (73, 1)]

In [45]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [46]:
new_document = "pounds mother sure think year rochester thorfield fairfax live elizabeth darcy bingley"

new_tokens = tokenize(new_document)

new_bow = id2word.doc2bow(new_tokens)

new_topic = lda[new_bow]

print(new_topic)

[(8, 0.7524468), (9, 0.17532337)]


In [47]:
lda.print_topics()

[(0,
  '0.008*"bretton" + 0.006*"madame" + 0.005*"like" + 0.005*"little" + 0.003*"jane" + 0.003*"night" + 0.003*"good" + 0.003*"old" + 0.003*"come" + 0.003*"know"'),
 (1,
  '0.008*"miss" + 0.007*"dear" + 0.006*"little" + 0.005*"harriet" + 0.004*"knightley" + 0.004*"helen" + 0.004*"day" + 0.004*"great" + 0.004*"emma" + 0.004*"burns"'),
 (2,
  '0.026*"emma" + 0.018*"harriet" + 0.014*"elton" + 0.013*"miss" + 0.011*"knightley" + 0.011*"weston" + 0.011*"woodhouse" + 0.009*"think" + 0.008*"thing" + 0.008*"good"'),
 (3,
  '0.009*"madame" + 0.005*"pelet" + 0.004*"vous" + 0.004*"day" + 0.004*"monsieur" + 0.004*"little" + 0.003*"knew" + 0.003*"room" + 0.003*"school" + 0.003*"like"'),
 (4,
  '0.006*"little" + 0.006*"madame" + 0.005*"like" + 0.004*"day" + 0.004*"thought" + 0.004*"long" + 0.004*"night" + 0.004*"door" + 0.003*"good" + 0.003*"eyes"'),
 (5,
  '0.005*"little" + 0.004*"madame" + 0.004*"think" + 0.003*"room" + 0.003*"hand" + 0.003*"dr" + 0.003*"good" + 0.003*"thought" + 0.003*"paul" + 0.

In [48]:
words = [re.findall('"([^"]*)"',t[1]) for t in lda.print_topics()]

In [49]:
topics = [' '.join(t[0:5]) for t in words]

In [50]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
bretton madame like little jane

------ Topic 1 ------
miss dear little harriet knightley

------ Topic 2 ------
emma harriet elton miss knightley

------ Topic 3 ------
madame pelet vous day monsieur

------ Topic 4 ------
little madame like day thought

------ Topic 5 ------
little madame think room hand

------ Topic 6 ------
like little face thought saw

------ Topic 7 ------
like night time day thought

------ Topic 8 ------
miss elinor elizabeth know time

------ Topic 9 ------
like little john know miss

------ Topic 10 ------
sir little yes think jane

------ Topic 11 ------
little like thought room time

------ Topic 12 ------
little like bessie miss good

------ Topic 13 ------
like pelet crimsworth hunsden face

------ Topic 14 ------
graham polly little papa bretton



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [51]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [52]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.163496 -0.021995       1        1  32.724068
9      0.031749  0.022664       2        1  14.701414
4     -0.043217  0.099920       3        1  12.661407
7      0.004573  0.049187       4        1   7.946792
2      0.186245 -0.019638       5        1   7.933255
11     0.004057  0.039830       6        1   7.341459
12     0.007104  0.042971       7        1   7.001857
6     -0.035262  0.051267       8        1   2.348451
5     -0.050379  0.056197       9        1   1.854536
3     -0.077907  0.024056      10        1   1.524276
1      0.070513 -0.075957      11        1   1.254290
14    -0.023785 -0.016746      12        1   1.117541
0     -0.061323 -0.016658      13        1   1.010937
10    -0.074013 -0.119803      14        1   0.349901
13    -0.101851 -0.115295      15        1   0.229822, topic_info=         Term         Freq        Total Category  logprob  loglift
95       emma   917.000000   917.000000  Default  30.0000  30.0000
185    little  1690.000000  1690.000000  Default  29.0000  29.0000
1214     miss  1543.000000  1543.000000  Default  28.0000  28.0000
141   harriet   555.000000   555.000000  Default  27.0000  27.0000
1284      sir   581.000000   581.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
649     laugh     0.945901   121.287926  Topic15  -6.6906   1.2218
190   looking     0.961329   364.405365  Topic15  -6.6744   0.1379
439      mind     0.965777   566.074951  Topic15  -6.6698  -0.2979
1682   reason     0.940949   233.833405  Topic15  -6.6959   0.5601
133      gone     0.942851   341.095154  Topic15  -6.6938   0.1846

[1231 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6781       4  0.083549    abandoned
6781       7  0.668392    abandoned
6781       8  0.083549    abandoned
6781      10  0.083549    abandoned
10280      6  0.847858  abandonment
...      ...       ...          ...
6666       6  0.320941      zoraide
6666       7  0.240706      zoraide
6666       9  0.040118      zoraide
6666      10  0.200588      zoraide
6666      15  0.040118      zoraide

[4152 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 5, 8, 3, 12, 13, 7, 6, 4, 2, 15, 1, 11, 14])

### Overall Model / Documents

In [53]:
corpus[0][:50]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 2),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 2),
 (28, 1),
 (29, 2),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 2),
 (48, 1),
 (49, 1)]

In [54]:
lda[corpus[0]]

[(2, 0.1878781), (8, 0.53647065), (9, 0.19182312), (11, 0.08196765)]

In [55]:
distro = [lda[d] for d in corpus]

In [73]:
distro[0]

[(10, 0.13784067), (11, 0.7754888), (13, 0.084567785)]

In [56]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [57]:
len(new_distro)

813

In [58]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[giving, fair, companion, account, yesterday, ...",1,Emma,26
Austen_Emma0032,"[feels, like, snow, place, party, try, day, di...",1,Emma,32
CBronte_Villette0086,"[pierced, opaque, blackness, stood, bougie, qu...",0,Villette,86
CBronte_Jane0099,"[joke, queer, looks, tell, thing, specially, s...",0,Jane,99
CBronte_Villette0092,"[second, landing, floor, comprising, abode, kn...",0,Villette,92


In [59]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [60]:
df.head()

,bretton madame like little jane,miss dear little harriet knightley,emma harriet elton miss knightley,madame pelet vous day monsieur,little madame like day thought,little madame think room hand,like little face thought saw,like night time day thought,miss elinor elizabeth know time,like little john know miss,sir little yes think jane,little like thought room time,little like bessie miss good,like pelet crimsworth hunsden face,graham polly little papa bretton,author
Austen_Emma0026,0.0,0.0,0.187879,0.0,0.000000,0.0,0.0,0.000000,0.536479,0.191817,0.0,0.081965,0.0,0.0,0.0,Austen
Austen_Emma0032,0.0,0.0,0.564344,0.0,0.000000,0.0,0.0,0.000000,0.367587,0.065922,0.0,0.000000,0.0,0.0,0.0,Austen
CBronte_Villette0086,0.0,0.0,0.615721,0.0,0.381102,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,CBronte
CBronte_Jane0099,0.0,0.0,0.000000,0.0,0.071399,0.0,0.0,0.283082,0.000000,0.000000,0.0,0.643616,0.0,0.0,0.0,CBronte
CBronte_Villette0092,0.0,0.0,0.018084,0.0,0.292093,0.0,0.0,0.000000,0.080176,0.502513,0.0,0.105731,0.0,0.0,0.0,CBronte


In [61]:
df.groupby('author').mean()

,bretton madame like little jane,miss dear little harriet knightley,emma harriet elton miss knightley,madame pelet vous day monsieur,little madame like day thought,little madame think room hand,like little face thought saw,like night time day thought,miss elinor elizabeth know time,like little john know miss,sir little yes think jane,little like thought room time,little like bessie miss good,like pelet crimsworth hunsden face,graham polly little papa bretton
author,,,,,,,,,,,,,,,
Austen,0.000000,0.012995,0.160635,0.000557,0.000552,0.000089,0.000034,0.012971,0.741434,0.036994,0.000181,0.023254,0.005999,0.000136,0.001663
CBronte,0.016844,0.011870,0.014084,0.024177,0.213800,0.032318,0.040925,0.132805,0.015254,0.241274,0.005734,0.104575,0.122967,0.003598,0.017372


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [62]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=5, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=10)

In [82]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [83]:
topic_coherence.head()

,pass,num_topics,coherence_score
0,0,5,-0.638831
1,0,7,-0.698094
2,0,9,-0.714129
3,0,11,-0.692684
4,0,13,-0.758045


In [ ]:
import seaborn as sns
 
ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)